In [6]:
pip install --upgrade pip

  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 11.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 7.3 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
import catboost
print(catboost.__version__)

1.1.1


In [12]:
from catboost import CatBoostClassifier
from catboost import datasets

train_df, test_df = datasets.amazon() # nice datasets with categorical features only :D
train_df.shape, test_df.shape

((32769, 10), (58921, 10))

In [13]:
train_df.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [14]:
test_df.head()

,id,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,78766,72734,118079,118080,117878,117879,118177,19721,117880
1,2,40644,4378,117961,118327,118507,118863,122008,118398,118865
2,3,75443,2395,117961,118300,119488,118172,301534,249618,118175
3,4,43219,19986,117961,118225,118403,120773,136187,118960,120774
4,5,42093,50015,117961,118343,119598,118422,300136,118424,118425


In [15]:
y = train_df['ACTION']
X = train_df.drop(columns='ACTION') # or X = train_df.drop('ACTION', axis=1)

In [16]:
X_test = test_df.drop(columns='id')

In [17]:
SEED = 1
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=SEED)

In [19]:
pip install ipywidgets

  Using cached ipywidgets-8.0.2-py3-none-any.whl (134 kB)
  Using cached jupyterlab_widgets-3.0.3-py3-none-any.whl (384 kB)
  Using cached widgetsnbextension-4.0.3-py3-none-any.whl (2.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [20]:
%%time

params = {'loss_function':'Logloss', # objective function
          'eval_metric':'AUC', # metric
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': SEED
         }
cbc_1 = CatBoostClassifier(**params)
cbc_1.fit(X_train, y_train, # data to train on (required parameters, unless we provide X as a pool object, will be shown below)
          eval_set=(X_valid, y_valid), # data to validate on
          use_best_model=True, # True if we don't want to save trees created after iteration with the best validation score
          plot=True # True for visualization of the training process (it is not shown in a published kernel - try executing this code)
         );

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.069882
0:	test: 0.5400959	best: 0.5400959 (0)	total: 62.2ms	remaining: 1m 2s
200:	test: 0.8020842	best: 0.8020842 (200)	total: 1.13s	remaining: 4.49s
400:	test: 0.8237941	best: 0.8237941 (400)	total: 2.24s	remaining: 3.35s
600:	test: 0.8328464	best: 0.8330283 (598)	total: 3.3s	remaining: 2.19s
800:	test: 0.8366271	best: 0.8370599 (785)	total: 4.47s	remaining: 1.11s
999:	test: 0.8417823	best: 0.8417823 (999)	total: 5.61s	remaining: 0us

bestTest = 0.8417823185
bestIteration = 999

CPU times: user 10.7 s, sys: 6.33 s, total: 17 s
Wall time: 5.82 s


In [21]:
cat_features = list(range(X.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [22]:
cat_features_names = X.columns # here we specify names of categorical features
cat_features = [X.columns.get_loc(col) for col in cat_features_names]
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [23]:
condition = True # here we specify what condition should be satisfied only by the names of categorical features
cat_features_names = [col for col in X.columns if condition]
cat_features = [X.columns.get_loc(col) for col in cat_features_names]
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [24]:
%%time

params = {'loss_function':'Logloss',
          'eval_metric':'AUC',
          'cat_features': cat_features,
          'verbose': 200,
          'random_seed': SEED
         }
cbc_2 = CatBoostClassifier(**params)
cbc_2.fit(X_train, y_train,
          eval_set=(X_valid, y_valid),
          use_best_model=True,
          plot=True
         );

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.069882
0:	test: 0.5637606	best: 0.5637606 (0)	total: 13.9ms	remaining: 13.9s
200:	test: 0.8959353	best: 0.8959406 (199)	total: 3.15s	remaining: 12.5s
400:	test: 0.8997237	best: 0.8998715 (388)	total: 6.38s	remaining: 9.54s
600:	test: 0.9008757	best: 0.9014912 (505)	total: 9.56s	remaining: 6.34s
800:	test: 0.8987706	best: 0.9014912 (505)	total: 12.7s	remaining: 3.17s
999:	test: 0.8981788	best: 0.9014912 (505)	total: 15.9s	remaining: 0us

bestTest = 0.9014912028
bestIteration = 505

Shrink model to first 506 iterations.
CPU times: user 1min 8s, sys: 13.8 s, total: 1min 21s
Wall time: 16.1 s


early_stopping_rounds

In [25]:
%%time

params = {'loss_function':'Logloss',
          'eval_metric':'AUC',
          'cat_features': cat_features,
          'early_stopping_rounds': 200,
          'verbose': 200,
          'random_seed': SEED
         }
cbc_2 = CatBoostClassifier(**params)
cbc_2.fit(X_train, y_train, 
          eval_set=(X_valid, y_valid), 
          use_best_model=True, 
          plot=True
         );

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.069882
0:	test: 0.5637606	best: 0.5637606 (0)	total: 18.4ms	remaining: 18.4s
200:	test: 0.8959353	best: 0.8959406 (199)	total: 3.23s	remaining: 12.9s
400:	test: 0.8997237	best: 0.8998715 (388)	total: 6.45s	remaining: 9.63s
600:	test: 0.9008757	best: 0.9014912 (505)	total: 9.76s	remaining: 6.48s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9014912028
bestIteration = 505

Shrink model to first 506 iterations.
CPU times: user 47.7 s, sys: 9.53 s, total: 57.2 s
Wall time: 11.6 s


task_type=GPU ->  GPU 가 있는경우

In [26]:
%%time

params = {'loss_function':'Logloss',
          'eval_metric':'AUC',
          'cat_features': cat_features,
          'task_type': 'GPU',
          'verbose': 200,
          'random_seed': SEED
         }
cbc_3 = CatBoostClassifier(**params)
cbc_3.fit(X_train, y_train,
          eval_set=(X_valid, y_valid), 
          use_best_model=True,
          plot=True
         );

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CatBoostError: catboost/libs/train_lib/trainer_env.cpp:9: Environment for task type [GPU] not found